## **<span style="color:#023e8a;">Intro</span>**

**<span style="color:#023e8a;">The competition is dedicated to the product recomendations (H&M)  </span>**

**<span style="color:#023e8a;">Here we have different kinds of data that help us to get good recomendations: </span>**

📸 `images` - images of every article_id

🙋 `articles`  - detailed metadata of every article_id

👔 `customers`  - detailed metadata of every customer_id

🧾 `transactions_train`  - purchases with details

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import sys
import time

import logging
import threading

import math

import csv

## **<span id="Articles" style="color:#023e8a;">2. Articles</span>**

**<span style="color:#023e8a;"> This table contains all h&m articles with details such as a type of product, a color, a product group and other features.</span>**  
**<span style="color:#023e8a;"> Article data description: </span>**

> `article_id` **<span style="color:#023e8a;">: A unique identifier of every article.</span>**  
> `product_code`, `prod_name` **<span style="color:#023e8a;">: A unique identifier of every product and its name (not the same).</span>**  
> `product_type`, `product_type_name` **<span style="color:#023e8a;">: The group of product_code and its name</span>**  
> `graphical_appearance_no`, `graphical_appearance_name` **<span style="color:#023e8a;">: The group of graphics and its name</span>**  
> `colour_group_code`, `colour_group_name` **<span style="color:#023e8a;">: The group of color and its name</span>**  
> `graphical_appearance_no`, `graphical_appearance_name` **<span style="color:#023e8a;">: The group of graphics and its name</span>**  
> `perceived_colour_value_id`, `perceived_colour_value_name`, `perceived_colour_master_id`, `perceived_colour_master_name` **<span style="color:#023e8a;">: The added color info</span>**  
> `department_no`, `department_name`: **<span style="color:#023e8a;">: A unique identifier of every dep and its name</span>**  
> `index_code`, `index_name`: **<span style="color:#023e8a;">: A unique identifier of every index and its name</span>**  
> `index_group_no`, `index_group_name`: **<span style="color:#023e8a;">: A group of indeces and its name</span>**  
> `section_no`, `section_name`: **<span style="color:#023e8a;">: A unique identifier of every section and its name</span>**  
> `garment_group_no`, `garment_group_name`: **<span style="color:#023e8a;">: A unique identifier of every garment and its name</span>**  
> `detail_desc`: **<span style="color:#023e8a;">: Details</span>**  

**<span style="color:#023e8a;"> Customers data description: </span>**

> `customer_id` **<span style="color:#023e8a;">: A unique identifier of every customer</span>**  
> `FN` **<span style="color:#023e8a;">: 1 or missed </span>**  
> `Active` **<span style="color:#023e8a;">: 1 or missed</span>**  
> `club_member_status` **<span style="color:#023e8a;">: Status in club</span>**  
> `fashion_news_frequency` **<span style="color:#023e8a;">: How often H&M may send news to customer</span>**  
> `age` **<span style="color:#023e8a;">: The current age</span>**  
> `postal_code` **<span style="color:#023e8a;">: Postal code of customer</span>**  

## **<span id="Transactions" style="color:#023e8a;">4. Transactions</span>**

[**<span style="color:#FEF1FE;background-color:#023e8a;border-radius: 5px;padding: 2px">Go to Table of Content</span>**](#Content)

**<span style="color:#023e8a;"> Transactions data description: </span>**

> `t_dat` **<span style="color:#023e8a;">: A unique identifier of every customer</span>**  
> `customer_id` **<span style="color:#023e8a;">: A unique identifier of every customer </span>**  **<span style="color:#FF0000;">(in </span>** `customers` **<span style="color:#FF0000;"> table)</span>**  
> `article_id` **<span style="color:#023e8a;">: A unique identifier of every article</span>**  **<span style="color:#FF0000;">(in </span>** `articles` **<span style="color:#FF0000;"> table)</span>**  
> `price` **<span style="color:#023e8a;">: Price of purchase</span>**  
> `sales_channel_id` **<span style="color:#023e8a;">: 1 or 2</span>**  

In [14]:
class ProgressBar:
    
    def __init__(self, transactions, trans_len = None):
        self.percent_intervals = 3
        self.percent_rounded_decimals = 2
        if trans_len == None:
            self.trans_len = len(transactions)
        else:
            self.trans_len = trans_len
        self.percent = int(self.trans_len / 10**self.percent_intervals)
        self.epsilon = 10**-2
        self.start = time.time()

    def check(self, i, chunked=False):
        if i % self.percent == 0 or i + 1 == self.trans_len or chunked:
            end = time.time()
            percent_decimal = i / (self.trans_len - 1)
            percent_current = percent_decimal * 100
            time_elapsed = end - self.start
            time_estimated = time_elapsed / (percent_decimal + self.epsilon)
            sys.stdout.write("%6.2f%% time elapsed: %d, estimated: %d\r" % (percent_current, time_elapsed, time_estimated))

In [146]:
length = 31788325
#count = 0
#with open("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", newline='') as csvfile:
#    spamreader = csv.reader(csvfile, delimiter=',')
#    for row in spamreader:
#        count += 1
#        if count % 1000000 == 0:
#            print(count)
#print(count)
length1 = 1371981 #sample_submission length and customers.csv length
#count = 0
#with open("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv", newline='') as csvfile:
#    spamreader = csv.reader(csvfile, delimiter=',')
#    print(spamreader)

#    for row in spamreader:
#        count += 1
#        if count % 10000000 == 0:
#            print(count)
#print(count)

In [149]:
customer_hash_age = {}
pb = ProgressBar([],length1)
with open("../input/h-and-m-personalized-fashion-recommendations/customers.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    print(spamreader)
    count = -1
    for row in spamreader:
        count += 1
        pb.check(count)
        if count == 0:
            print(row)
            continue
        customer_id = row[0]
        age = row[5]
        customer_hash_age[customer_id] = age

['customer_id', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'postal_code']


In [21]:
customer_hash = {}
pb = ProgressBar([],length)
with open("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    print(spamreader)
    count = -1
    for row in spamreader:
        count += 1
        pb.check(count)
        if count == 0:
            print(row)
            continue
        customer_id = row[1]
        article_id = row[2]
        if customer_id not in customer_hash:
            customer_hash[customer_id] = []
            
        customer_hash[customer_id].append(article_id)

['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']


In [37]:
customer_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']

['0663713001',
 '0541518023',
 '0663713001',
 '0578020002',
 '0723529001',
 '0351484002',
 '0351484002',
 '0727808001',
 '0727808007',
 '0858883002',
 '0851400006',
 '0750424014',
 '0750424014',
 '0870304002',
 '0870304002',
 '0852643001',
 '0852643003',
 '0794321007']

In [27]:
# open the file in the write mode
f = open('customer_hash.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(['article_ids', 'customer_id'])

keys = customer_hash.keys()
pb = ProgressBar([],len(keys))
i = -1
for customer_id in keys:
    i += 1
    pb.check(i)
    article_ids = customer_hash[customer_id]
    article_ids.append(customer_id)
    
    writer.writerow(article_ids)
# close the file
f.close()

In [ ]:
customer_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']

In [36]:
keys = customer_hash.keys()
pb = ProgressBar([],len(keys))
i = -1
for customer_id in keys:
    i += 1
    pb.check(i)
    article_ids = customer_hash[customer_id]
    customer_hash[customer_id] = article_ids[0:len(article_ids)-1]

In [38]:
customer_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']

['0663713001',
 '0541518023',
 '0663713001',
 '0578020002',
 '0723529001',
 '0351484002',
 '0351484002',
 '0727808001',
 '0727808007',
 '0858883002',
 '0851400006',
 '0750424014',
 '0750424014',
 '0870304002',
 '0870304002',
 '0852643001',
 '0852643003',
 '0794321007']

In [98]:
maxC = 0
for customer_id in customer_hash.keys():
    maxC1 = len(customer_hash[customer_id])
    if maxC1 > maxC:
        print(customer_id)
        maxC = maxC1

000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2
003ac6eff3ca20cbe4e14a106a5e28b7551616005de8f9dd0f9455e5543b31ac
00609a1cc562140fa87a6de432bef9c9f0b936b259ad3075eb2a65008df1dbab
01a9a077a83eacc6fd946a60c4ca2aed16219fcd83d31e3110fa04630c42b96d
01e464bf74b13a55df22de1528eff2b33749c0cd92953b62bd22dee2de17d1fd
03d0011487606c37c1b1ed147fc72f285a50c05f00b9712e0fc3da400c864296
49beaacac0c7801c2ce2d189efe525fe80b5d37e46ed05b50a4cd88e34d0748f
be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b


In [102]:
maxC#, customer_hash['be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b']

1895

In [104]:
article_count = {}
for article_id in customer_hash['be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee985513d9e8e53c6d91b']:
    if article_id not in article_count:
        article_count[article_id] = 0
    article_count[article_id] += 1
maxCC = 0
for article_id in article_count.keys():
    count = article_count[article_id]
    if maxCC < count:
        maxCC = count
        print(article_id, count)

0658506001 3
0668956001 7
0253448001 8


In [132]:
#table = customers_hash
table = train_customers
customer_hash_hash = {}
substitutive = 0
substitutive_uniq = 0
pb = ProgressBar([], len(table.keys()))
k = -1
for customer_id in table.keys():
    k += 1
    pb.check(k)
    articles = table[customer_id]
    found = False
    article_hash = {}
    for article_id in articles:
        if article_id not in article_hash:
            article_hash[article_id] = 0
        article_hash[article_id] += 1
        
    for article_id in article_hash.keys():
        count = article_hash[article_id]
        if count > 1:
            substitutive += count
            if not found:
                substitutive_uniq += 1
                found = True
    customer_hash_hash[customer_id] = article_hash
substitutive, substitutive_uniq

(7114662, 613976)

In [133]:
substitutive_uniq/len(customer_hash.keys()) # 0.49207542349926336 people rebuy clothes

0.45069702946748874

In [150]:
customer_hash_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']['0663713001']

2

In [155]:
len(customer_hash_hash.keys()) / len(customer_hash_age.keys()), len(customer_hash_age.keys()) -len(customer_hash_hash.keys())

(0.9929306549658158, 9699)

In [159]:
notin = []
pb = ProgressBar([], len(customer_hash_age.keys()))
i = -1
for customer_id in customer_hash_age.keys():
    i += 1
    pb.check(i)
    if customer_id in customer_hash_hash:
        continue
    notin.append(customer_id)

In [171]:
len(notin),notin[0]

(9699, '00058ecf091cea1bba9d800cabac6ed1ae284202cdab68bec5c8429eb3271c0c')

In [162]:
pb = ProgressBar([], len(notin))
i = -1
for customer_id in notin:
    i += 1
    pb.check(i)
    customer_hash_hash[customer_id] = {}
    customer_hash_hash[customer_id]['0663713001'] = 1

In [163]:
def sort_obj_by_keys(obj, sort_on_keys=True, reverse=False):
    
    keys = []
    values = []
    for key in obj.keys():
        keys.append(key)
        values.append(obj[key])
    
    ind = None
    if sort_on_keys:
        ind = np.argsort(keys)
    else:
        ind = np.argsort(values)
    
    keys_sort = np.array(keys)[ind]
    values_sort = np.array(values)[ind]
    
    if reverse:
        keys_sort = np.flip(keys_sort)
        values_sort = np.flip(values_sort)
    
    newObj = {}
    newObj['keys'] = keys_sort
    newObj['values'] = values_sort
    
    return newObj

In [164]:
sort_obj_by_keys(customer_hash_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'], False, True)

{'keys': array(['0870304002', '0750424014', '0351484002', '0663713001',
        '0852643003', '0852643001', '0851400006', '0858883002',
        '0727808007', '0727808001', '0723529001', '0578020002',
        '0541518023'], dtype='<U10'),
 'values': array([2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [165]:
customer_hash_hash_sorted = {}
pb = ProgressBar([], len(customer_hash_hash.keys()))
i = -1
for customer_id in customer_hash_hash.keys():
    i += 1
    pb.check(i)
    customer_hash_hash_sorted[customer_id] = sort_obj_by_keys(customer_hash_hash[customer_id], False, True)

In [166]:
customer_hash_hash_sorted['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']['values'][0]

2

In [170]:
len(customer_hash_hash_sorted.keys()) - len(customer_hash_age.keys())

0

In [167]:
import csv

# open the file in the write mode
f = open('submission-purely-substitutive.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(['customer_id','prediction'])

pb = ProgressBar([], len(customer_hash_hash_sorted.keys()))
i = -1
for customer_id in customer_hash_hash_sorted:
    i += 1
    pb.check(i)
    articles = customer_hash_hash_sorted[customer_id]['keys']
    article_ids = ''
    
    end = 10
    if len(articles) < 10:
        end = len(articles)
    for k in range(0,end):
        article_id = str(articles[k])
        for l in range(len(article_id), 10):
            article_id = '0' + article_id
        article_ids += str(article_id) + ' '
    writer.writerow([customer_id, article_ids])
# close the file
f.close()

In [41]:
minC = maxC
for customer_id in customer_hash.keys():
    minC1 = len(customer_hash[customer_id])
    if minC1 < minC:
        minC = minC1

In [42]:
minC

1

In [43]:
minCC = 0
for customer_id in customer_hash.keys():
    minCC1 = len(customer_hash[customer_id])
    if minCC1 > 24:
        minCC += 1

In [44]:
minCC / len(customer_hash.keys())

0.2700213832535285

In [45]:
customer_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'][0]

'0663713001'

In [46]:
customer_hash['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'][0:10]

['0663713001',
 '0541518023',
 '0663713001',
 '0578020002',
 '0723529001',
 '0351484002',
 '0351484002',
 '0727808001',
 '0727808007',
 '0858883002']

In [49]:
train_customers = {}
test_customers = {}
test_cutoff = 0.1
pb = ProgressBar([], len(customer_hash.keys()))
count = -1
for customer_id in customer_hash.keys():
    count += 1
    pb.check(count)
    
    articles1 = customer_hash[customer_id]
    length = len(articles1)
    test_count = int(length * test_cutoff)
    if test_count == 0:
        test_count = 1
    
    
    train_list = []
    test_list = []
    for i in range(length):
        article = articles1[i]
        if (i < length - test_count):
            train_list.append(article)
        else:
            test_list.append(article)
            
    train_customers[customer_id] = train_list
    test_customers[customer_id] = test_list
    

In [50]:
train_customers['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']

['0663713001',
 '0541518023',
 '0663713001',
 '0578020002',
 '0723529001',
 '0351484002',
 '0351484002',
 '0727808001',
 '0727808007',
 '0858883002',
 '0851400006',
 '0750424014',
 '0750424014',
 '0870304002',
 '0870304002',
 '0852643001',
 '0852643003']

In [51]:
test_customers['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318']

['0794321007']

In [52]:
keys = train_customers.keys()

In [53]:
import csv

# open the file in the write mode
f = open('train_customer_hash.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(['article_ids','customer_id'])

pb = ProgressBar([], len(train_customers.keys()))
i = -1
for customer_id in train_customers:
    i += 1
    pb.check(i)
    articles = train_customers[customer_id]
    articles.append(customer_id)
    writer.writerow(articles)
    pb.check(i)
# close the file
f.close()

In [56]:
keys = train_customers.keys()
pb = ProgressBar([],len(keys))
i = -1
for customer_id in keys:
    i += 1
    pb.check(i)
    article_ids = train_customers[customer_id]
    train_customers[customer_id] = article_ids[0:len(article_ids)-1]

In [54]:
import csv

# open the file in the write mode
f = open('test_customer_hash.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(['article_ids','customer_id'])

pb = ProgressBar([], len(test_customers.keys()))
i = -1
for customer_id in test_customers:
    i += 1
    pb.check(i)
    articles = test_customers[customer_id]
    articles.append(customer_id)
    writer.writerow(articles)
    pb.check(i)
# close the file
f.close()

In [59]:
keys = test_customers.keys()
pb = ProgressBar([],len(keys))
i = -1
for customer_id in keys:
    i += 1
    pb.check(i)
    article_ids = test_customers[customer_id]
    test_customers[customer_id] = article_ids[0:len(article_ids)-1]

In [61]:
#test_customers

In [95]:
len(customer_hash.keys()) # 1362281

1362281

In [92]:
len(train_customers.keys()) # 1362281

1362281

In [91]:
len(test_customers.keys()) # 1362281

1362281

In [74]:
test_customers['00058ecf091cea1bba9d800cabac6ed1ae284202cdab68bec5c8429eb3271c0c']

KeyError: '00058ecf091cea1bba9d800cabac6ed1ae284202cdab68bec5c8429eb3271c0c'

In [76]:
#test_customers

In [83]:
count_rights = []
pb = ProgressBar([],length1)
with open("../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    print(spamreader)
    count = -1
    for row in spamreader:
        count += 1
        pb.check(count)
        if count == 0:
            print(row)
            continue
        if count == 1:
            print(row)
            print(row[1].split(' '))
        customer_id = row[0]
        if customer_id not in test_customers:
            continue
        test_articles = test_customers[customer_id]
        count_right = 0
        for article_id in row[1].split(' '):
            for test_article_id in test_articles:
                if article_id == test_article_id:
                    count_right += 1
        count_rights.append(count_right / len(test_articles))

['customer_id', 'prediction']mated: 0
['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657', '0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001']
['0706016001', '0706016002', '0372860001', '0610776002', '0759871002', '0464297007', '0372860002', '0610776001', '0399223001', '0706016003', '0720125001', '0156231001']


In [84]:
#count_rights
len(count_rights), length1

(1362281, 1371981)

In [85]:
maxC = 0
for count_right in count_rights:
    if maxC < count_right:
        maxC = count_right
maxC

1.0

In [86]:
avgC = 0
for count_right in count_rights:
    avgC += count_right
avgC = avgC / len(count_rights)
avgC
#0.01087662574725043 sample_subission on test_customers dataset

0.01087662574725043

In [87]:
count_rights = []
pb = ProgressBar([],length1)
with open("../input/hm-submission-5/submission-5.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    print(spamreader)
    count = -1
    for row in spamreader:
        count += 1
        pb.check(count)
        if count == 0:
            print(row)
            continue
        if count == 1:
            print(row)
            print(row[1].split(' '))
        customer_id = row[0]
        if customer_id not in test_customers:
            continue
        test_articles = test_customers[customer_id]
        count_right = 0
        for article_id in row[1].split(' '):
            for test_article_id in test_articles:
                if article_id == test_article_id:
                    count_right += 1
        count_rights.append(count_right / len(test_articles))

['customer_id', 'prediction']mated: 1
['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657', '0751471001 0751471043 0924243001 0915529003 0751471038 0918522001 0673677002 0806131012 0909370001 0706016003 ']
['0751471001', '0751471043', '0924243001', '0915529003', '0751471038', '0918522001', '0673677002', '0806131012', '0909370001', '0706016003', '']


In [88]:
#count_rights
len(count_rights), length1

(1362281, 1371981)

In [89]:
maxC = 0
for count_right in count_rights:
    if maxC < count_right:
        maxC = count_right
maxC

1.0

In [90]:
avgC = 0
for count_right in count_rights:
    avgC += count_right
avgC = avgC / len(count_rights)
avgC
#0.005721774062530521 submission 5 on test_customers dataset
#0.01087662574725043 sample_subission on test_customers dataset

0.005721774062530521

In [140]:
count_rights = []
pb = ProgressBar([],length1)
with open("/kaggle/working/submission-purely-substitutive.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    print(spamreader)
    count = -1
    for row in spamreader:
        count += 1
        pb.check(count)
        if count == 0:
            print(row)
            continue
        if count == 1:
            print(row)
            print(row[1].split(' '))
        customer_id = row[0]
        if customer_id not in test_customers:
            continue
        test_articles = test_customers[customer_id]
        count_right = 0
        for article_id in row[1].split(' '):
            for test_article_id in test_articles:
                if article_id == test_article_id:
                    count_right += 1
        count_rights.append(count_right / len(test_articles))

['customer_id', 'prediction']mated: 0
['000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318', '0870304002 0750424014 0351484002 0663713001 0852643003 0852643001 0851400006 0858883002 0727808007 0727808001 ']
['0870304002', '0750424014', '0351484002', '0663713001', '0852643003', '0852643001', '0851400006', '0858883002', '0727808007', '0727808001', '']


In [141]:
#count_rights
len(count_rights), length1

(1362281, 1371981)

In [142]:
maxC = 0
for count_right in count_rights:
    if maxC < count_right:
        maxC = count_right
maxC

1.0

In [143]:
avgC = 0
for count_right in count_rights:
    avgC += count_right
avgC = avgC / len(count_rights)
avgC
#0.005721774062530521 submission 5 on test_customers dataset
#0.01087662574725043 sample_subission on test_customers dataset
#0.8081491059035482 submission purely substitutive (incorrect since test data was trained on)
#0.09428860753617259 submission purely substitutive (no test data leakage, so this is a valid score)

0.09428860753617259